## Imports and useful funcs

In [1]:
runs_folder = 'Macro_TESA_Lin5'



In [2]:
from torch.utils.data import DataLoader
from Solvation_1.my_nets.Create_dataset import *
from Solvation_1.Vectorizers.vectorizers import *
from Solvation_1.my_nets.LinearNet import *
from Solvation_1.my_nets.net_func import *

## Create dataset

In [3]:
table_v1 = pd.read_csv(project_path('Solvation_1/Tables/table_SS_v1.csv'))
dataset = SS_Dataset(table_v1, 'solvent_macro_props1', 'solute_TESA')
len_data = dataset.__len__()
val_data = len_data // 10
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [len_data - val_data, val_data])

solvent_table = pd.read_table(project_path('Solvation_1/Tables/solvent_test_table_v1.tsv'))
solute_table = pd.read_table(project_path('Solvation_1/Tables/solute_test_table_v1.tsv'))
solvent_test_dataset = SS_Dataset(solvent_table, 'solvent_macro_props1', 'solute_TESA')
solute_test_dataset = SS_Dataset(solute_table, 'solvent_macro_props1', 'solute_TESA')

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

solvent_test_loader = DataLoader(solvent_test_dataset, batch_size=46, shuffle=False)
solute_test_loader = DataLoader(solute_test_dataset, batch_size=64, shuffle=False)

## Train Network

In [4]:
x, y = next(iter(dataset))
x.shape

torch.Size([1, 16])

In [5]:
in_feat = next(iter(dataset))[0].shape[-1]
verbose = False
model = LinearNet3(in_features = in_feat)
loss_function = nn.MSELoss(reduction='sum')
optimizer = torch.optim.Adam(model.parameters())
model.train()

MSE = train(model,train_loader, val_loader, solvent_test_loader, solute_test_loader, loss_function, optimizer, epochs=12, ckp_path=runs_folder)

100%|██████████| 25/25 [00:00<00:00, 27.58it/s]


/Users/balepka/PycharmProjects/msuAI/Solvation_1/../Solvation_1/Runs/Macro_TESA_Lin5/ep_0.pt
Validation loss decreased (10000000000000000.000000 --> 2.694940).  Saving model ...
/Users/balepka/PycharmProjects/msuAI/Solvation_1/../Solvation_1/Runs/Macro_TESA_Lin5/ep_0.pt
epoch 0 -> 2.6949400901794434


100%|██████████| 25/25 [00:00<00:00, 26.35it/s]


/Users/balepka/PycharmProjects/msuAI/Solvation_1/../Solvation_1/Runs/Macro_TESA_Lin5/ep_5.pt
Validation loss decreased (2.694940 --> 1.460458).  Saving model ...
/Users/balepka/PycharmProjects/msuAI/Solvation_1/../Solvation_1/Runs/Macro_TESA_Lin5/ep_5.pt
epoch 5 -> 1.460457682609558


100%|██████████| 25/25 [00:00<00:00, 25.29it/s]


/Users/balepka/PycharmProjects/msuAI/Solvation_1/../Solvation_1/Runs/Macro_TESA_Lin5/ep_10.pt
Validation loss decreased (1.460458 --> 0.943209).  Saving model ...
/Users/balepka/PycharmProjects/msuAI/Solvation_1/../Solvation_1/Runs/Macro_TESA_Lin5/ep_10.pt
epoch 10 -> 0.9432094097137451


100%|██████████| 25/25 [00:00<00:00, 31.60it/s]


In [6]:
len(train_loader.dataset)

1571